In [1]:
import torch
import wandb
import pprint
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy as dpcp
import pandas as pd 
import wandb
import os
import itertools
os.environ["http_proxy"]="http://proxy61.iitd.ac.in:3128"
os.environ["https_proxy"]="http://proxy61.iitd.ac.in:3128"
if torch.cuda.is_available():
  device = torch.device('cuda', index = 0)
else:
  device = 'cpu'

In [2]:
def get_cross_table(versions, models, datasets, metric):
  api = wandb.Api()
  runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")
  data_names = [dat.split("-")[1] for dat in datasets]
  return_runs = []
  df = pd.DataFrame({f'{model}-{ver}': [torch.nan for _ in datasets] for ver, model in zip(versions,models)}, index = data_names)
  for (ver, model), dat in itertools.product(zip(versions, models), datasets):
    for run in runs:
      # print(run.config['start_time'])
      if run.state == 'finished' and run.config['model'] == model and dat == run.config['dataset'] and ver == run.config['run_name'] and metric in run.summary:
          df.loc[dat.split("-")[1], f'{model}-{ver}'] = round(100 * run.summary[metric], 2)
          break
  return df

In [4]:
# versions = ['old', 'old', 'old', 'mxmdcorr', 'mxmdraw', 'mxmd-normal']
# models = ['TGAT', 'TGN' , 'DyGFormer', 'TGN', 'TGN', 'TGN']
versions = ['old', 'old', 'old', 'old', 'mxmd-v3', 'mxmd-linear']
models = ['TGAT', 'JODIE', 'TGN', 'DyGFormer',  'TGN', 'TGN']
datasets = ["ia-digg-reply", "ia-escorts-dynamic", "ia-retweet-pol", "ia-reality-call", "ia-slashdot-reply-dir", "ia-movielens-user2tags-10m"]
df = get_cross_table(versions, models, datasets, 'test average_precision')

In [5]:
df

,TGAT-old,JODIE-old,TGN-old,DyGFormer-old,TGN-mxmd-v3,TGN-mxmd-linear
digg,64.32,52.72,59.45,63.50,63.44,63.62
escorts,59.57,60.91,63.14,61.70,57.28,59.23
retweet,53.69,57.13,67.86,58.26,52.64,54.37
reality,72.47,82.78,90.55,91.08,88.49,87.69
slashdot,87.80,64.99,90.97,86.78,89.22,88.32
movielens,61.63,68.09,69.98,63.20,57.48,55.11


In [3]:
# versions = ['old', 'old', 'old', 'mxmdcorr', 'mxmdraw', 'mxmd-normal']
# models = ['TGAT', 'TGN' , 'DyGFormer', 'TGN', 'TGN', 'TGN']
versions = ['old-32dim-hist', 'old-historical', 'old-64dim-hist', 'mxmd-32dim-hist', 'mxmd-rnd-historical', 'mxmd-64dim-hist', 'mxmd-64dim-diff-hist']
models = ['TGN', 'TGN', 'TGN', 'TGN', 'TGN', 'TGN', 'TGN']
datasets = ["ia-digg-reply", "ia-escorts-dynamic", "ia-retweet-pol", "ia-reality-call", "ia-slashdot-reply-dir", "ia-movielens-user2tags-10m"]
df = get_cross_table(versions, models, datasets, 'test average_precision')
df

,TGN-old-32dim-hist,TGN-old-historical,TGN-old-64dim-hist,TGN-mxmd-32dim-hist,TGN-mxmd-rnd-historical,TGN-mxmd-64dim-hist,TGN-mxmd-64dim-diff-hist
digg,80.12,81.93,81.21,82.21,83.96,83.00,83.00
escorts,86.82,89.36,87.71,88.09,88.15,88.66,88.66
retweet,85.70,NaN,85.97,81.17,86.22,84.18,84.18
reality,95.50,97.74,96.38,87.15,97.87,94.49,94.49
slashdot,89.28,91.11,90.34,93.32,93.54,93.54,93.54
movielens,89.07,89.93,89.44,88.56,90.04,87.24,87.24


In [7]:
versions = ['old-historical', 'old-historical', 'mxmd-rnd-historical']
models = ['TGN', 'DyGFormer', 'TGN']
datasets = ["ia-digg-reply", "ia-escorts-dynamic", "ia-retweet-pol", "ia-reality-call", "ia-slashdot-reply-dir", "ia-movielens-user2tags-10m"]
df = get_cross_table(versions, models, datasets, 'test average_precision')
df

,TGN-old-historical,DyGFormer-old-historical,TGN-mxmd-rnd-historical
digg,81.93,74.80,83.96
escorts,89.36,86.51,88.15
retweet,86.70,85.16,86.22
reality,97.74,99.28,97.87
slashdot,91.11,92.26,93.54
movielens,89.93,92.44,90.04


In [9]:
df = get_cross_table(versions, models, datasets, 'new node test first_3_average_precision')
df

,TGN-old-historical,DyGFormer-old-historical,TGN-mxmd-rnd-historical
digg,62.19,60.91,64.33
escorts,65.49,65.89,59.50
retweet,52.81,59.59,54.50
reality,90.27,93.61,89.36
slashdot,86.40,85.20,87.04
movielens,65.91,69.24,66.29


In [4]:
versions = ['old-inductive', 'old-inductive', 'mxmd-rnd-inductive']
models = ['TGN', 'DyGFormer', 'TGN']
datasets = ["ia-digg-reply", "ia-escorts-dynamic", "ia-retweet-pol", "ia-reality-call", "ia-slashdot-reply-dir", "ia-movielens-user2tags-10m"]
df = get_cross_table(versions, models, datasets, 'test average_precision')
df

,TGN-old-inductive,DyGFormer-old-inductive,TGN-mxmd-rnd-inductive
digg,81.93,74.80,83.96
escorts,89.36,86.51,88.15
retweet,86.70,88.93,86.22
reality,97.74,99.28,97.87
slashdot,91.11,92.21,93.54
movielens,89.93,NaN,90.04


In [4]:
import pandas as pd

def df_to_latex_table(df, caption="Your Caption Here", label="tab:your_label"):
    # Start the table
    latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{|l|" + "c|"*len(df.columns) + "}\n\\hline\n"
    
    # Add header
    header = " & ".join(["\\textbf{" + col + "}" for col in df.columns])
    latex_table += "\\textbf{Dataset} & " + header + " \\\\ \\hline\n"
    
    # Add data rows
    for index, row in df.iterrows():
        row_data = " & ".join([f"{x:.2f}" if pd.notnull(x) else "---" for x in row])
        latex_table += f"{index} & {row_data} \\\\ \n"
    
    # Close the table
    latex_table += "\\end{tabular}\n"
    latex_table += f"\\caption{{{caption}}}\n"
    latex_table += f"\\label{{{label}}}\n"
    latex_table += "\\end{table}"
    
    return latex_table


# Convert to LaTeX table
latex_table = df_to_latex_table(df, caption="Model Performance Comparison", label="tab:model_performance")

# Print the LaTeX table
print(latex_table)


\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|}
\hline
\textbf{Dataset} & \textbf{TGN-smallsplit-old} & \textbf{TGN-smallsplit-mxmd} \\ \hline
digg & 80.66 & 78.16 \\ 
escorts & 87.49 & 88.90 \\ 
retweet & 85.23 & 83.44 \\ 
reality & 97.38 & 96.35 \\ 
slashdot & 91.29 & 91.15 \\ 
movielens & 85.88 & 86.29 \\ 
\end{tabular}
\caption{Model Performance Comparison}
\label{tab:model_performance}
\end{table}


In [40]:
df1 = get_cross_table(versions, models, datasets, 'new node test average_precision')
df1

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,65.67,67.21,65.52,64.09,68.35,NaN
escorts,NaN,74.03,76.18,NaN,NaN,67.85
retweet,68.46,76.50,75.87,74.05,75.66,69.10
reality,86.78,95.22,97.73,94.92,94.93,NaN
slashdot,83.87,87.50,88.72,86.24,87.43,NaN
movielens,NaN,81.89,84.98,NaN,NaN,77.03


In [59]:
df2 = get_cross_table(versions, models, datasets, 'new node test first_1_average_precision')
df2

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,63.21,60.26,63.23,66.43,62.34,NaN
escorts,NaN,61.53,62.08,NaN,NaN,61.91
retweet,53.30,68.50,58.19,64.95,67.71,53.50
reality,74.17,89.28,91.19,83.85,84.49,NaN
slashdot,85.67,89.46,86.83,87.06,88.20,NaN
movielens,NaN,66.38,65.54,NaN,NaN,56.87


In [60]:
df2 = get_cross_table(versions, models, datasets, 'new node test first_3_average_precision')
df2

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,62.79,62.19,63.11,61.41,63.89,NaN
escorts,NaN,65.71,66.09,NaN,NaN,60.51
retweet,56.40,66.97,63.15,63.93,65.66,57.58
reality,77.29,90.27,93.78,88.69,89.25,NaN
slashdot,81.19,86.40,86.23,84.21,85.18,NaN
movielens,NaN,66.55,69.14,NaN,NaN,63.88


In [44]:
api = wandb.Api()
runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")

In [56]:
run =runs[0]
run.start_time

AttributeError: '<Run fb-graph-proj/fb-graph-proj-dyglib/hy0wo06a (finished)>' object has no attribute 'start_time'